In [1]:
import os, sys
sys.path.append('/home/dcooper/git/Bridgeport')
from interactive.interactive_utils import *
from utils.utils import write_FASTA
from Bridgeport.Bridgeport import Bridgeport
import py3Dmol
from datetime import datetime
from rdkit.Chem.Draw import rdDepictor

/home/dcooper/anaconda3/envs/prep/lib/python3.12/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


**Let's start with some basic information...**

**Protein Input**

- *input_pdb_dir*: (str) path to directory where the *input_pdb* can be found. 
- *input_pdb*: (str) name of input structure
- *chain*: (str) of chain to use from *input_pdb* to align

**Ligand Input** (if you are doing an analogue, fill in this information for the templace ligand in your experimental structure)

- *name*: (str) a name for your ligand
- *resname*: (str) resname of ligand found in *input_pdb*. If the ligand, is a peptide set to "False".
- *chainid*: (str) if ligand is a peptide, specify the letter code that denotes the ligand, if not set to "False".
- *smiles*: (str) smiles string for you ligand. 

In [2]:
# Set you working directory
input_dir = '/home/dcooper/projects/GIP/'

In [3]:
# Protein input
input_pdb_dir = os.path.join(input_dir, 'input_pdb') # Change as needed
input_pdb = 'tripeptide.pdb' # Change as needed
chain = 'R' # Change as needed


# Ligand input
name = 'triagonist'
resname = False
chainid = 'P'


In [4]:
# Build initial .json file and BP object
working_dir = '/home/dcooper/projects/GIP'
json_dict = {'working_dir': working_dir}

# Update params
json_dict['Protein'] = {'input_pdb_dir': input_pdb_dir,
                        'input_pdb': input_pdb,
                        'chain': chain}


json_dict['Ligand'] = {'name': name,
                       'resname': resname,
                       'chainid': chainid}
json_fn = 'interactive.json'
write_json(json_dict, json_fn)
start = datetime.now()
BP = Bridgeport(json_fn, verbose=False)

07/09/2025 23:01:24//Welcome to Bridgeport.
07/09/2025 23:01:24//Found input parameters.
07/09/2025 23:01:24//working_dir:
07/09/2025 23:01:24//working_dir: /home/dcooper/projects/GIP
07/09/2025 23:01:24//Protein:
07/09/2025 23:01:24//	input_pdb_dir: /home/dcooper/projects/GIP/input_pdb
07/09/2025 23:01:24//	input_pdb: tripeptide.pdb
07/09/2025 23:01:24//	chain: R
07/09/2025 23:01:24//Ligand:
07/09/2025 23:01:24//	name: triagonist
07/09/2025 23:01:24//	resname: False
07/09/2025 23:01:24//	chainid: P


1. **Align your protein to a reference structure**
- *reference_pdb*: (str) path to refence structure to align input structure to
- *reference_chain*: (List[str]) of chains of the *reference_pdb* to use in the alignment process

In [5]:
# Set input paths
reference_pdb = os.path.join(input_dir, 'OPM', '7ra3.pdb') # Change as needed
reference_chain = ['R'] # Change as needed

# Update input params
BP.input_params['Environment'] = {'alignment_ref': reference_pdb,
                                  'reference_chain': reference_chain}

# Align
BP.align_to_reference()

# Visualizes aligned structures
view = py3Dmol.view()
view.setBackgroundColor('white')
view.addModel(open(BP.aligned_pdb, 'r').read(),'pdb')
view.addModel(open(reference_pdb, 'r').read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'blue'}})
view.setStyle({'model':1}, {'cartoon': {'color':'yellow'}})
view.zoomTo()
view.show()


07/09/2025 23:01:24//Found directory for aligned input structures: /home/dcooper/projects/GIP/aligned_input_pdb
07/09/2025 23:01:24//Found references structure /home/dcooper/projects/GIP/OPM/7ra3.pdb and will align to chains ['R']
07/09/2025 23:01:24//Found input structure: /home/dcooper/projects/GIP/input_pdb/tripeptide.pdb
07/09/2025 23:01:24//Saved aligned structure to: /home/dcooper/projects/GIP/aligned_input_pdb/triagonist.pdb


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### ANALOGUE

In [6]:
# # # # Analogue input
# name = 'noradrenaline' # Change as needed
# smiles = 'NC[C@H](O)c1cc(O)c(O)cc1' # Change as needed
# add_atoms = False
# remove_atoms = False # Change as needed
# # Update params
# BP.input_params['Ligand']['Analogue'] = {'name': name, 'smiles': smiles, 'add_atoms': add_atoms, 'remove_atoms': remove_atoms}
# BP.get_analogue_MCS()

In [7]:
# # Inputs
# align_all = True # Change as needed
# rmsd_thresh = 1 # Change as needed
# n_conformers = 1

# # Update params
# BP.input_params['Ligand']['Analogue']['align_all'] = align_all
# BP.input_params['Ligand']['Analogue']['rmsd_thresh'] = rmsd_thresh
# BP.input_params['Ligand']['Analogue']['n_conformers'] = n_conformers

# BP.build_analogue_complex() 
# BP.analogue.visualize_alignment()

2. **Seperate your ligand and protein for individual preparations**

In [8]:
# Separate
BP.separate_lig_prot()

07/09/2025 23:01:24//Found directory for protein structures: /home/dcooper/projects/GIP/proteins
07/09/2025 23:01:24//Found directory for ligand structures: /home/dcooper/projects/GIP/ligands
07/09/2025 23:01:24//Separated chain(s) R from input structure
07/09/2025 23:01:24//Separated ligand P from input structure


3. **Repair the protein**
   - *tails*: List of indices to parse the extra tails. EX: [30, 479]
   - *loops*: 2-D List of indices that specify lower and upper bounds of loops to optimize during refinement. Loop optimization can take a while, but if skipped, unbonded output structures will result.
   - *secondary_template*: Path to secondary .pdb to use as a reference to accurately model large portions that are missing in the input .pdb structure.
   - *engineered_resids*: List of resids that are known engineered mutations in the crystal pdb. Adding this argument may prevent sequence errors in the RepairProtein section of Bridgeport.
   - *receptor_gene*: (str) gene of the receptor
   - *sequence*: (str) sequence of the receptor you are trying to model

In [9]:
# # Set input
# tails = [29, 411] # Change as needed
# loops = False # Change as needed
# secondary_template = None #os.path.join(input_pdb_dir, 'GIPR_AF.pdb') # Change as needed
# engineered_resids = None # Change as needed
# receptor_gene = 'GIP' # Change as needed
# sequence = """
# MTTSPILQLLLRLSLCGLLLQRAETGSKGQTAGELYQRWERYRRECQETLAAAEPPSGLACNGSFDMYVCWDYAAPNATARASCPWYLPWHHHVAAGFVLRQCGSDGQWGLWRDHTQCENPEKNEAFLDQRLILERLQVMYTVGYSLSLATLLLALLILSLFRRLHCTRNYIHINLFTSFMLRAAAILSRDRLLPRPGPYLGDQALALWNQALAACRTAQIVTQYCVGANYTWLLVEGVYLHSLLVLVGGSEEGHFRYYLLLGWGAPALFVIPWVIVRYLYENTQCWERNEVKAIWWIIRTPILMTILINFLIFIRILGILLSKLRTRQMRCRDYRLRLARSTLTLVPLLGVHEVVFAPVTEEQARGALRFAKLGFEIFLSSFQGFLVSVLYCFINKEVQSEIRRGWHHCRLRRSLGEEQRQLPERAFRALPSGSGPGEVPTSRGLSSGTLPGPGNEASRELESYC
# """ # Change as needed

# # Write FASTA
# fasta_path = os.path.join(input_dir, 'fasta', f'{receptor_gene}.fasta')
# write_FASTA(sequence, receptor_gene, fasta_path)

# # Update params
# BP.input_params['RepairProtein'] = {'fasta_path': fasta_path,
#                                     'working_dir': os.path.join(os.getcwd(), 'modeller_intermediates'),
#                                     'tails': tails, 
#                                     'loops': loops,
#                                     'secondary_template': secondary_template,
#                                     'engineered_resids': engineered_resids}
# # Repair
# BP.repair_protein()

# # Visualizes repaired protein
# view = py3Dmol.view()
# view.setBackgroundColor('white')
# view.addModel(open(os.path.join(BP.prot_only_dir, BP.name + '.pdb'), 'r').read(),'pdb')
# view.setStyle({'model':0}, {'cartoon': {'color':'blue'}})
# view.zoomTo()
# view.show()

4. **Build the environment**

- "membrane": If membrane should be specified choose "true", and make sure that "alignment_ref" argument is the appropriate OPM structure. Default is false.
- "pH": Specify the pH. Default is 7.0.
- "ion_strength": Specify the concentration of NaCl ions (in Molar). Default is 0.15 M.


In [10]:
# # Set input
# membrane = True # Change as needed
# pH = 7.2 # Change as needed
# ion_strength = 0.15 # Change as needed

# BP.input_params['Environment']['membrane'] = membrane
# BP.input_params['Environment']['pH'] = pH
# BP.input_params['Environment']['ion_strength'] = ion_strength

# BP.add_environment()

5. **Prepare the ligand**
- *small_molecule_params (bool)*: If true, treat ligand like a small molecule. Default is True.
- *sanitize (bool)*: If true, sanitize molecule with rdkit. Default is True. Only applicable if small_molecule_params is True. 
- *removeHs (bool)*: If true, remove any hydrogens that may be present. Default is True. Only applicable if small_molecule_params is True.
- *proximityBonding* (bool): If true, use rdkit's 'proximityBonding' method to load rdkit molecule. 
- *pH (float)*: pH to protonate a peptide ligand. Default is 7.0.
- *nstd_resids (List[int])*: List of nonstandard resids to conserve from input structure. 
- *neutral_Cterm (bool)*: If true, neutralize the C-terminus of a peptide ligand. Only applicable is small_molecule_params is False


In [11]:
# Ligand Input
sequence = 'HAQGTFTSDKSKYLDERAAQDFVQWLLDGGPSSGAPPPS'
nstd_resids = None # Change as needed
pH = 7.0 # Change as needed
chain = 'P'

# Update params
BP.input_params['Ligand']['nstd_resids'] = nstd_resids
BP.input_params['Ligand']['pH'] = pH
BP.input_params['Ligand']['chain'] = chain
BP.input_params['Ligand']['sequence'] = sequence
BP.input_params['Ligand']['MutatedPeptide'] = {}
BP.input_params['Ligand']['MutatedPeptide']['Mutations'] = []

BP.input_params['Ligand']['MutatedPeptide']['Mutations'].append({})
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['mutation_resid'] = 2
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['mutation_resname'] = 'AIB'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['mutation_smiles'] = 'CC(C)(NC=O)C(N)=O'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['remove_atoms'] = ['N2', 'H9', 'H10', 'C4', 'O1', 'H8']
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['change_atoms'] = [{'C5': 'protein-C', 'O2': 'protein-O'}, {'N1': 'protein-N'}]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['bonds_to_add'] = [[[1, 'C'], [2, 'N1']], [[2, 'C5'], [3, 'N']]]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][0]['external_bonds'] = ['N1', 'C5']

BP.input_params['Ligand']['MutatedPeptide']['Mutations'].append({})
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['mutation_resid'] = 10
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['mutation_resname'] = 'KAC'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['mutation_smiles'] = 'CCCCCCCCCCCCCCCC(=O)N[C@@H](CCC(=O)NCCCC[C@H](NC=O)C(N)=O)C(O)=O'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['remove_atoms'] = ['N4', 'H50', 'H51', 'C26', 'O3', 'H49']
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['change_atoms'] = [{'C27': 'protein-C', 'O4': 'protein-O'}, {'N3': 'protein-N'}]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['bonds_to_add'] = [[[9, 'C'], [10, 'N3']], [[10, 'C27'], [11, 'N']]]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][1]['external_bonds'] = ['N3', 'C27']

BP.input_params['Ligand']['MutatedPeptide']['Mutations'].append({})
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['mutation_resid'] = 39
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['mutation_resname'] = 'NXS'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['mutation_smiles'] = 'NC(=O)[C@H](CO)NC=O'
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['remove_atoms'] = ['C4', 'O3', 'H8']
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['change_atoms'] = [{'N2': 'protein-N'}]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['bonds_to_add'] = [[[38, 'C'], [39, 'N2']]]
BP.input_params['Ligand']['MutatedPeptide']['Mutations'][2]['external_bonds'] = ['N2']

# Prepare
BP.ligand_prep()

# Visualizes ligand sdf and pdb
view = py3Dmol.view()
view.setBackgroundColor('white')
if hasattr(BP, 'lig_sdf'):
    view.addModel(open(BP.lig_sdf, 'r').read(),'sdf')
else:
    view.addModel(open(BP.lig_pdb, 'r').read(),'pdb')
view.setStyle({'model':0}, {'stick': {'colorscheme':'cyanCarbon'}})
view.zoomTo()
view.show()


07/09/2025 23:01:24//Found peptide ligand with resname: P
07/09/2025 23:01:24//Welcome to RepairProtein
07/09/2025 23:01:24//Protein to repair: /home/dcooper/projects/GIP/ligands/AIB2_reference.pdb
07/09/2025 23:01:24//Template sequence: /home/dcooper/projects/GIP/scripts/prep/lig.fasta
07/09/2025 23:01:24//Modeller intermediates will be written to: /home/dcooper/projects/GIP/scripts/prep/modeller
0 atoms in HETATM/BLK residues constrained
to protein atoms within 2.30 angstroms
and protein CA atoms within 10.00 angstroms
0 atoms in residues without defined topology
constrained to be rigid bodies

>> Summary of successfully produced models:
Filename                          molpdf
----------------------------------------
lig.B99990001.pdb               97.91580

07/09/2025 23:01:25//Moved protein from 8.294636092299987 to 1.0943096002022406
07/09/2025 23:01:25//Protein Repaired. Output written to: /home/dcooper/projects/GIP/ligands/AIB2_reference.pdb
07/09/2025 23:01:25//Welcome to Prot

INFO:PDB2PQR v3.6.1: biomolecular structure conversion software.
INFO:Please cite:  Jurrus E, et al.  Improvements to the APBS biomolecular solvation software suite.  Protein Sci 27 112-128 (2018).
INFO:Please cite:  Dolinsky TJ, et al.  PDB2PQR: expanding and upgrading automated preparation of biomolecular structures for molecular simulations. Nucleic Acids Res 35 W522-W525 (2007).
INFO:Checking and transforming input arguments.
INFO:Loading topology files.
INFO:Loading molecule: /home/dcooper/projects/GIP/ligands/AIB2_reference.pdb
ERROR:Error parsing line: invalid literal for int() with base 10: ''
ERROR:<REMARK     285 UNITARY VALUES FOR THE UNIT CELL AUTOMATICALLY SET>
ERROR:Truncating remaining errors for record type:REMARK

ERROR:['REMARK']
INFO:Setting up molecule.
INFO:Created biomolecule object with 39 residues and 295 atoms.
INFO:Setting termini states for biomolecule chains.
INFO:Loading forcefield.
INFO:Loading hydrogen topology definitions.
INFO:This biomolecule is clean.

07/09/2025 23:01:25//Saved prepared ligand to /home/dcooper/projects/GIP/ligands/AIB2_reference_residue.pdb /home/dcooper/projects/GIP/ligands/AIB2_reference_residue.sdf
07/09/2025 23:01:25//Created analogue AIB2_residue from smiles: CC(C)(NC=O)C(N)=O
07/09/2025 23:01:25//Created known ligand AIB2_reference_residue from smiles: N[C@@H](C)C=O
07/09/2025 23:01:25//Saved conformer to /home/dcooper/projects/GIP/ligands/AIB2_residue_conformers/AIB2_residue_0.pdb
07/09/2025 23:01:25//Saved first conformer to /home/dcooper/projects/GIP/ligands/AIB2_residue.pdb
07/09/2025 23:01:25//Saved prepared ligand to /home/dcooper/projects/GIP/ligands/AIB2_residue.pdb /home/dcooper/projects/GIP/ligands/AIB2_residue.sdf
Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for pdb File --
   Status: pass

-I: Adding /home/dcooper/anaconda3/envs

[23:01:25] WARNING: More than one matching pattern found - picking one



0.0 0 0.0
tot_chg 0.0
07/09/2025 23:01:26//Added residue HIS 1 to topology
07/09/2025 23:01:26//Added residue AIB 2 to topology
07/09/2025 23:01:26//Added residue GLN 3 to topology
07/09/2025 23:01:26//Added residue GLY 4 to topology
07/09/2025 23:01:26//Added residue THR 5 to topology
07/09/2025 23:01:26//Added residue PHE 6 to topology
07/09/2025 23:01:26//Added residue THR 7 to topology
07/09/2025 23:01:26//Added residue SER 8 to topology
07/09/2025 23:01:26//Added residue ASP 9 to topology
07/09/2025 23:01:26//Added residue LYS 10 to topology
07/09/2025 23:01:26//Added residue SER 11 to topology
07/09/2025 23:01:26//Added residue LYS 12 to topology
07/09/2025 23:01:26//Added residue TYR 13 to topology
07/09/2025 23:01:26//Added residue LEU 14 to topology
07/09/2025 23:01:26//Added residue ASP 15 to topology
07/09/2025 23:01:26//Added residue GLU 16 to topology
07/09/2025 23:01:26//Added residue ARG 17 to topology
07/09/2025 23:01:26//Added residue ALA 18 to topology
07/09/2025 23:0

07/09/2025 23:01:30//Saved conformer to /home/dcooper/projects/GIP/ligands/KAC10_residue_conformers/KAC10_residue_0.pdb
07/09/2025 23:01:30//Saved first conformer to /home/dcooper/projects/GIP/ligands/KAC10_residue.pdb
07/09/2025 23:01:30//Saved prepared ligand to /home/dcooper/projects/GIP/ligands/KAC10_residue.pdb /home/dcooper/projects/GIP/ligands/KAC10_residue.sdf
Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for pdb File --
   Status: pass

-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/prep to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/lib to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/parm to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/cmd to search path.
-f: Source /home/dcooper/projects/GIP/ligands/KAC10_residue.tleap.

Welcome to L

[23:01:30] WARNING: More than one matching pattern found - picking one



0.0 0 0.0
tot_chg 0.0
07/09/2025 23:01:30//Added residue HIS 1 to topology
07/09/2025 23:01:30//Added residue AIB 2 to topology
07/09/2025 23:01:30//Added residue GLN 3 to topology
07/09/2025 23:01:30//Added residue GLY 4 to topology
07/09/2025 23:01:30//Added residue THR 5 to topology
07/09/2025 23:01:30//Added residue PHE 6 to topology
07/09/2025 23:01:30//Added residue THR 7 to topology
07/09/2025 23:01:30//Added residue SER 8 to topology
07/09/2025 23:01:30//Added residue ASP 9 to topology
07/09/2025 23:01:30//Added residue KAC 10 to topology
07/09/2025 23:01:30//Added residue SER 11 to topology
07/09/2025 23:01:30//Added residue LYS 12 to topology
07/09/2025 23:01:30//Added residue TYR 13 to topology
07/09/2025 23:01:30//Added residue LEU 14 to topology
07/09/2025 23:01:30//Added residue ASP 15 to topology
07/09/2025 23:01:30//Added residue GLU 16 to topology
07/09/2025 23:01:30//Added residue ARG 17 to topology
07/09/2025 23:01:30//Added residue ALA 18 to topology
07/09/2025 23:0

Info: acdoctor mode is on: check and diagnose problems in the input file.
Info: The atom type is set to gaff; the options available to the -at flag are
      gaff, gaff2, amber, bcc, abcg2, and sybyl.

-- Check Format for pdb File --
   Status: pass

-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/prep to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/lib to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/parm to search path.
-I: Adding /home/dcooper/anaconda3/envs/prep/dat/leap/cmd to search path.
-f: Source /home/dcooper/projects/GIP/ligands/NXS39_residue.tleap.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: /home/dcooper/projects/GIP/ligands/NXS39_residue.tleap
----- Source: /home/dcooper/anaconda3/envs/prep/dat/leap/cmd/leaprc.gaff
----- Source of /home/dcooper/anaconda3/envs/prep/dat/leap/cmd/leaprc.gaff done
Log file: ./leap.log
Loading parameters: /home/dcooper/anaconda3/envs/prep/dat/leap/parm/gaff.dat
Reading title:
AMBER Ge

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

6. **Build the forcefields**


In [12]:
BP.env_pdb = '/home/dcooper/projects/GIP/proteins/triagonist_env.pdb'
BP.generate_systems()

07/09/2025 23:01:30//Building parameters for triagonist
07/09/2025 23:01:33//Protein parameters built.
07/09/2025 23:01:33//Ligand parameters built.
07/09/2025 23:01:34//System parameters built.
07/09/2025 23:01:34//Initial structure potential energy: 1672046863315992.2


**Save your input_parameters to a .json file**
- *json_fn*: str path to .json file where you would like to save your input parameters

In [13]:
# Change ligand smiles back
# BP.input_params['Ligand']['smiles'] = lig_smiles

# Set .json location
json_fn = os.path.join(input_dir, 'json', f'{name}.json')
write_json(BP.input_params, json_fn)

print('WROTE:', json_fn)
print('TOOK:', datetime.now() - start)

WROTE: /home/dcooper/projects/GIP/json/triagonist.json
TOOK: 0:00:11.583790


In [14]:
# from openmm.app import *

# pdb = PDBFile('/home/dcooper/projects/GIP/ligands/AIB2_reference.pdb')
# top = pdb.getTopology()
# for a in top.atoms():
#     print(a.index)